# Diffusion of obstacles

In [1]:
import numpy as np
import matplotlib.pyplot as plt


n_nucleo = 10
s_nucleo = 15

n_linker = n_nucleo + 1
s_linker = 5

s_shift = 1

def alpha_diffused(
    n_nucleo: int,
    s_nucleo: int,
    n_linker: int,
    s_linker: int,
    s_shift: int,
    alphaf: float,
    alphao: float
) -> np.ndarray :
    """Generate a 1D alpha profile representing a chromatin fiber composed of nucleosomes and linkers,
    with random diffusion (shifts) of linker regions around nucleosomes.

    Args:
        n_nucleo (int): Number of nucleosomes.
        s_nucleo (int): Size (in base pairs) of each nucleosome.
        n_linker (int): Number of linker regions (typically n_nucleo - 1).
        s_linker (int): Size (in base pairs) of each linker region.
        s_shift (int): Maximum extent (in bp) of the fuzzy linker region on each side of a nucleosome.
                       Must satisfy s_shift ≤ s_linker // 2 to prevent overlapping.
        alphaf (float): Alpha value assigned to fuzzy (linker) regions.
        alphao (float): Alpha value assigned to ordered (nucleosome) regions.

    Returns:
        np.ndarray | None: The resulting 1D alpha profile, or None if the given s_shift is too large.
    
    Notes:
        - The base profile alternates between linker and nucleosome regions with fixed periodicity.
        - Each nucleosome is randomly assigned a side (left or right) where additional linker fuzziness is applied.
        - The final profile is plotted: initial in blue, diffused in red.
    """
    
    if s_shift > s_linker // 2:
        print("No such s_shift value permitted because of possible entanglement")
        return None

    # Random shifts: -1 (right) or 1 (left) for each nucleosome
    # Construct the initial periodic alpha profile
    shifts = np.random.choice([-1, 1], size=n_nucleo, p=[0.5, 0.5])
    s_period = s_linker + s_nucleo
    shifted = np.array(
        (s_linker * [alphaf] + s_nucleo * [alphao]) * n_nucleo + s_linker * [alphaf],
        dtype=float
    )

    # Apply diffusion around each nucleosome
    for i in range(n_nucleo):
        nucleo_start = s_linker + i * s_period
        shifted[nucleo_start - s_shift : nucleo_start + s_nucleo + s_shift] = alphao

        if shifts[i] == 1:
            # Fuzzy region to the left
            shifted[nucleo_start - s_shift : nucleo_start + s_shift] = alphaf
        else:
            # Fuzzy region to the right
            shifted[nucleo_start - s_shift + s_nucleo : nucleo_start + s_shift + s_nucleo] = alphaf

    return shifted


alpha_diffused(n_nucleo, s_nucleo, n_linker, s_linker, s_shift, 1, 0)

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
       1.])

In [70]:
import numpy as np
from tls.utils import listoflist_into_matrix

x_matrix = np.array([
    [0, 5, 0, 10, 10, 15, 10, 20, 20],
    [0, 12, 0, 14, 14, np.nan, np.nan, np.nan, np.nan]
])
t_matrix = np.array([
    [0.0, 2.1, 3.4, 10.1, 16.0, 22.7, 34.2, 52.2, 52.3],
    [0.0, 3.7, 4.2, 6.1, 35.7, np.nan, np.nan, np.nan, np.nan]
])


def get_jumps(
    t_matrix: np.ndarray,
    x_matrix: np.ndarray,
    total_return: bool = False
) -> tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:

    # Formalism 1
    t_forward_all, x_forward_all = [], []
    t_reverse_all, x_reverse_all = [], []
    
    # Formalism 2
    if total_return:
        t_try_all, x_try_all = [], []
        t_rejected_all, x_rejected_all = [], []
        t_accepted_all, x_accepted_all = [], []
    
    # Loop parameters
    n_traj, n_steps = x_matrix.shape
    
    for i in range(n_traj):
        
        # Each Trajectory
        t_traj = t_matrix[i]
        x_traj = x_matrix[i]
        
        # Formalism 1
        t_forward, x_forward = [], [] 
        t_reverse, x_reverse = [], []
        
        # Formalism 2
        if total_return:
            t_try, x_try = [], [] 
            t_rejected, x_rejected = [], [] 
            t_accepted, x_accepted = [], [] 
        
        for j in range(len(t_traj) - 1):
            
            dx = x_traj[j+1] - x_traj[j]
            
            if dx < 0:
                t_reverse.append(t_traj[j+1] - t_traj[j-1])  
                x_reverse.append(x_traj[j])
                if total_return:
                    t_rejected.append(t_traj[j+1] - t_traj[j])                

            elif dx == 0:
                t_forward.append(t_traj[j+1] - t_traj[j-1])      
                x_forward.append(x_traj[j+1])
                if total_return:
                    t_accepted.append(t_traj[j+1] - t_traj[j])              
            
            elif dx > 0:
                if total_return:
                    t_try.append(t_traj[j+1] - t_traj[j])
                    x_try.append(x_traj[j+1])
                
        if total_return:       
            x_rejected = x_reverse
            x_accepted = x_forward
            
        t_forward_all.append(t_forward)
        x_forward_all.append(x_forward)
        t_reverse_all.append(t_reverse)        
        x_reverse_all.append(x_reverse)
        
        if total_return:
            t_try_all.append(t_forward)
            x_try_all.append(x_forward)
            t_rejected_all.append(t_rejected) 
            x_rejected_all.append(x_rejected) 
            t_accepted_all.append(t_accepted) 
            x_accepted_all.append(x_accepted) 
        
    if not total_return:
        return (
            listoflist_into_matrix(t_forward_all),
            listoflist_into_matrix(x_forward_all),
            listoflist_into_matrix(t_reverse_all),
            listoflist_into_matrix(x_reverse_all),
        )
        
    else:
        return (
            listoflist_into_matrix(t_forward_all),
            listoflist_into_matrix(x_forward_all),
            listoflist_into_matrix(t_reverse_all),
            listoflist_into_matrix(x_reverse_all),
            listoflist_into_matrix(t_try_all),
            listoflist_into_matrix(x_try_all),
            listoflist_into_matrix(t_rejected_all),
            listoflist_into_matrix(x_rejected_all),
            listoflist_into_matrix(t_accepted_all),
            listoflist_into_matrix(x_accepted_all),
        )


t_forward, x_forward, t_reverse, x_reverse = get_jumps(t_matrix, x_matrix)
print(t_forward)

print(np.cumsum(t_forward, axis=1))


[[12.6 18.1]
 [31.5  nan]]
[[12.6 30.7]
 [31.5  nan]]


In [1]:
print("***")

***


# .